In [1]:
# Import necessary libraries
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
from sklearn.metrics import classification_report
from pathlib import Path

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify directories
train_dir = "../data/Training"
test_dir = "../data/Testing"

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Split train dataset into training and validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained EfficientNet model
efficientnet_model = models.efficientnet_b0(pretrained=True)

# Freeze all layers except the classifier
for param in efficientnet_model.parameters():
    param.requires_grad = False

# Replace the classifier for 4 classes
num_features = efficientnet_model.classifier[1].in_features
efficientnet_model.classifier[1] = nn.Linear(num_features, 4)  # 4 classes
efficientnet_model = efficientnet_model.to(device)

# Only the classifier's parameters will be updated
optimizer = optim.Adam(efficientnet_model.classifier[1].parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Fine-tuning loop
epochs = 20  # Fine-tune for 20 epochs
for epoch in range(epochs):
    efficientnet_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = efficientnet_model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Save the fine-tuned model
checkpoint_path = "/temp/tumor-detection/checkpoints/efficientnet/efficientnet_full.pth"
#checkpoint_dir = os.path.dirname(checkpoint_path)
save_dir = os.path.expanduser('~') + checkpoint_path
#checkpoint_path = "./checkpoint/resnet/resnet_trained.weights.h5"
#checkpoint_dir = os.path.dirname(checkpoint_path)
torch.save(efficientnet_model, save_dir)
print("Fine-tuned model saved successfully!")

# Testing loop with classification report
efficientnet_model.eval()
test_correct = 0
test_total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = efficientnet_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        # Store predictions and labels for classification report
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")


/home/zzhen268/anaconda3/envs/ai2env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zzhen268/anaconda3/envs/ai2env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 0.6007, Accuracy: 81.46%
Epoch [2/20], Loss: 0.3770, Accuracy: 87.59%
Epoch [3/20], Loss: 0.3392, Accuracy: 88.42%
Epoch [4/20], Loss: 0.3082, Accuracy: 89.49%
Epoch [5/20], Loss: 0.2956, Accuracy: 90.00%
Epoch [6/20], Loss: 0.2772, Accuracy: 90.37%
Epoch [7/20], Loss: 0.2625, Accuracy: 90.90%
Epoch [8/20], Loss: 0.2514, Accuracy: 91.35%
Epoch [9/20], Loss: 0.2519, Accuracy: 91.18%
Epoch [10/20], Loss: 0.2576, Accuracy: 90.96%
Epoch [11/20], Loss: 0.2329, Accuracy: 91.68%
Epoch [12/20], Loss: 0.2341, Accuracy: 91.90%
Epoch [13/20], Loss: 0.2288, Accuracy: 91.64%
Epoch [14/20], Loss: 0.2323, Accuracy: 91.57%
Epoch [15/20], Loss: 0.2182, Accuracy: 92.41%
Epoch [16/20], Loss: 0.2233, Accuracy: 91.99%
Epoch [17/20], Loss: 0.2275, Accuracy: 92.14%
Epoch [18/20], Loss: 0.2186, Accuracy: 92.21%
Epoch [19/20], Loss: 0.2191, Accuracy: 92.12%
Epoch [20/20], Loss: 0.2196, Accuracy: 92.36%
Fine-tuned model saved successfully!
Test Accuracy: 91.38%


In [3]:
print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")

Epoch [20/20], Loss: 0.2196, Accuracy: 92.36%
Test Accuracy: 91.38%


In [5]:
# Generate classification report
report = classification_report(all_labels, all_preds, target_names=train_dataset.dataset.classes)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

      glioma       0.97      0.82      0.89       300
  meningioma       0.81      0.86      0.83       306
     notumor       0.94      0.99      0.97       405
   pituitary       0.94      0.96      0.95       300

    accuracy                           0.91      1311
   macro avg       0.91      0.91      0.91      1311
weighted avg       0.92      0.91      0.91      1311



In [2]:
import os
checkpoint_path = "/temp/tumor-detection/checkpoints/efficientnet/efficientnet_full.pth"
#checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_parent_path = "/temp/tumor-detection/checkpoints/efficientnet/"
parent_dir = os.path.expanduser('~') + ckpt_parent_path
Path(parent_dir).mkdir(parents=True, exist_ok = True)
save_dir = os.path.expanduser('~') + checkpoint_path
#checkpoint_path = "./checkpoint/resnet/resnet_trained.weights.h5"
#checkpoint_dir = os.path.dirname(checkpoint_path)
torch.save(efficientnet_model, save_dir)

In [8]:
torch.save({
            'epoch': epoch,
            'model_state_dict': efficientnet_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss,
            }, os.path.expanduser('~') + checkpoint_path + "_full")

In [11]:
torch.save(optimizer.state_dict(), os.path.expanduser('~') + checkpoint_path + "_opt")